In [2]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.types import StructType, StructField, BooleanType, StringType, IntegerType, DateType, FloatType,DoubleType,ArrayType,LongType
import logging
import sys
import traceback
import ast
from pyspark.sql.functions import length

import json
import os
import sys
import traceback
import logging
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr,when,to_date ,udf, concat_ws,posexplode, from_json
from pyspark.sql.types import StructType, StructField, BooleanType, StringType, IntegerType, DateType, FloatType,DoubleType
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, expr
from pyspark.sql.types import ArrayType, FloatType
from sparknlp.base import DocumentAssembler, Pipeline
from sparknlp.annotator import (
    Tokenizer,
    Normalizer,
    StopWordsCleaner,
    BertEmbeddings,
    SentenceEmbeddings,DistilBertEmbeddings
)
from sparknlp.base import DocumentAssembler, Pipeline
import numpy as np
import faiss
import os
import boto3

In [3]:
# spark = SparkSession.builder \
#     .appName("MinIO with Delta Lake BERT") \
#     .config("spark.driver.memory", "8g") \
#     .config("spark.executor.memory", "8g") \
#     .config("spark.driver.cores", "6") \
#     .config("spark.executor.cores", "6") \
#     .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3") \
#     .config("spark.dynamicAllocation.enabled", "true") \
#     .config("spark.dynamicAllocation.minExecutors", "1") \
#     .config("spark.dynamicAllocation.maxExecutors", "1") \
#     .config("spark.dynamicAllocation.initialExecutors", "1") \
#     .config("spark.sql.shuffle.partitions", "12") \
#     .config("spark.memory.offHeap.enabled", "true") \
#     .config("spark.memory.offHeap.size", "4g") \
#     .config("spark.jars", "jars/hadoop-aws-3.3.4.jar,jars/spark-sql-kafka-0-10_2.12-3.2.1.jar,jars/aws-java-sdk-bundle-1.12.262.jar,jars/delta-core_2.12-2.2.0.jar,jars/delta-storage-2.2.0.jar") \
#     .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
#     .config("spark.hadoop.fs.s3a.access.key", "conbo123") \
#     .config("spark.hadoop.fs.s3a.secret.key", "123conbo") \
#     .config("spark.hadoop.fs.s3a.path.style.access", "true") \
#     .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
#     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
#     .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
#     .config("delta.enable-non-concurrent-writes", "true") \
#     .config('spark.sql.warehouse.dir', "s3a://lakehouse/") \
#     .config("spark.sql.shuffle.partitions", "100") \
#     .getOrCreate()

spark = SparkSession.builder \
    .appName("HybridRecommenderMaxPerformance") \
    .config("spark.executor.memory", "6g") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.cores", "4") \
    .config("spark.executor.instances", "1") \
    .config("spark.default.parallelism", "8") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "4g") \
    .config("spark.jars", "jars/hadoop-aws-3.3.4.jar,jars/spark-sql-kafka-0-10_2.12-3.2.1.jar,jars/aws-java-sdk-bundle-1.12.262.jar,jars/delta-core_2.12-2.2.0.jar,jars/spark-nlp_2.12-5.1.3.jar,jars/delta-storage-2.2.0.jar") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "conbo123") \
    .config("spark.hadoop.fs.s3a.secret.key", "123conbo") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
    .config("delta.enable-non-concurrent-writes", "true") \
    .config("spark.sql.warehouse.dir", "s3a://lakehouse/") \
    .getOrCreate()

25/06/15 15:32:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark_df = spark.read.format("parquet").load("s3a://lakehouse/gold/machineDataBert")

25/06/15 15:28:00 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [4]:
from pyspark.ml import PipelineModel

nlp = PipelineModel.load("s3a://lakehouse/bert_pipeline1000")

25/06/15 15:32:08 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


Py4JJavaError: An error occurred while calling o83.load.
: java.lang.NoClassDefFoundError: ai/onnxruntime/OrtEnvironment
	at com.johnsnowlabs.ml.onnx.OnnxWrapper$.com$johnsnowlabs$ml$onnx$OnnxWrapper$$withSafeOnnxModelLoader(OnnxWrapper.scala:86)
	at com.johnsnowlabs.ml.onnx.OnnxWrapper$.read(OnnxWrapper.scala:131)
	at com.johnsnowlabs.ml.onnx.ReadOnnxModel.readOnnxModel(OnnxSerializeModel.scala:101)
	at com.johnsnowlabs.ml.onnx.ReadOnnxModel.readOnnxModel$(OnnxSerializeModel.scala:74)
	at com.johnsnowlabs.nlp.embeddings.DistilBertEmbeddings$.readOnnxModel(DistilBertEmbeddings.scala:493)
	at com.johnsnowlabs.nlp.embeddings.ReadDistilBertDLModel.readModel(DistilBertEmbeddings.scala:438)
	at com.johnsnowlabs.nlp.embeddings.ReadDistilBertDLModel.readModel$(DistilBertEmbeddings.scala:429)
	at com.johnsnowlabs.nlp.embeddings.DistilBertEmbeddings$.readModel(DistilBertEmbeddings.scala:493)
	at com.johnsnowlabs.nlp.embeddings.ReadDistilBertDLModel.$anonfun$$init$$1(DistilBertEmbeddings.scala:4

In [6]:
import numpy as np
import faiss
from pyspark.sql.functions import col

vec_df = spark.read.format("delta").load("s3a://lakehouse/data/bert2")
rows = vec_df.select("id", "vecs").rdd.map(lambda row: (row["id"], np.array(row["vecs"], dtype=np.float32))).collect()

ids = [r[0] for r in rows]
vectors = np.stack([r[1] for r in rows])

norms = np.linalg.norm(vectors, axis=1, keepdims=True)
normalized_vectors = vectors / norms

index = faiss.IndexFlatIP(normalized_vectors.shape[1])
index.add(normalized_vectors)

print(f"✅ FAISS Cosine Index chứa {index.ntotal} vectors.")

25/06/15 13:19:39 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


✅ FAISS Cosine Index chứa 20658 vectors.


In [13]:
import numpy as np
import faiss
from pyspark.sql.functions import col

# Truy vấn người dùng nhwos nhập chổ này
##
#
user_query = "superhero save the earth"



query_df = spark.createDataFrame([[user_query]], ["comb_bert"])

# Lấy vector
query_vector = nlp.transform(query_df) \
    .select(col("sentence_embeddings.embeddings")[0].alias("vec")) \
    .collect()[0]["vec"]
query_vector = np.array(query_vector, dtype=np.float32)

# Normalize
query_vector /= np.linalg.norm(query_vector)
query_vector = query_vector.reshape(1, -1)

# FAISS search
k = 10
scores, indices = index.search(query_vector, k)

# In kết quả
print("🎯 Top kết quả (Cosine Similarity):")
matched_ids = []
for i, idx in enumerate(indices[0]):
    matched_ids.append(ids[idx])
    print(f"{i+1}. Movie ID: {ids[idx]}, Cosine Score: {scores[0][i]:.4f}")

# Lọc kết quả từ DataFrame
results = spark_df.filter(col("id").isin(matched_ids)) \
                  .select("id", "comb_bert")

results.show(10,False)

🎯 Top kết quả (Cosine Similarity):
1. Movie ID: 177271, Cosine Score: 0.7915
2. Movie ID: 17445, Cosine Score: 0.7694
3. Movie ID: 1924, Cosine Score: 0.7691
4. Movie ID: 103269, Cosine Score: 0.7639
5. Movie ID: 72693, Cosine Score: 0.7617
6. Movie ID: 283995, Cosine Score: 0.7596
7. Movie ID: 323027, Cosine Score: 0.7593
8. Movie ID: 436969, Cosine Score: 0.7585
9. Movie ID: 16237, Cosine Score: 0.7573
10. Movie ID: 3604, Cosine Score: 0.7551
+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------